In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

%matplotlib inline

In [2]:
uniq_id_cust= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_customers_dataset.csv')
orders= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_orders_dataset.csv')
items= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_order_items_dataset.csv')

6. Построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию
R - время от последней покупки пользователя до текущей даты
F - суммарное количество покупок у пользователя за всё время
M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. 
Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.  

In [3]:
#объединим таблицы
join_item_orders= items.merge(orders, how = 'outer', on ='order_id')
all= join_item_orders.merge(uniq_id_cust, how = 'outer', on ='customer_id')

In [4]:
all.head ()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [5]:
# приведем  к формуту даты
all['order_approved_at'] = pd.to_datetime(all['order_approved_at'])

In [6]:
# Проверим на отсутствие ID
print('{:,} без наименования Id'.format(all[all.customer_id.isnull()].shape[0]))

0 без наименования Id


In [7]:
# Посмотрим на временные рамки
print('Заказы с {} по {}'.format(all['order_approved_at'].min(),
                                    all['order_approved_at'].max()))

Заказы с 2016-09-15 12:16:38 по 2018-09-03 17:40:06


In [8]:
# Уберем из выборки созданные, выставлен счет (неоплачен) недоступные и отмененные 
all = all\
    .query('order_status != ["unavailable", "created", "invoiced", "canceled"]')\

In [9]:
all.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP


In [10]:
# перевели в год, в котором сделан заказ и оплачен
all['year_approved'] = all.order_approved_at.dt.year

In [11]:
# Так как обычно для рассчета CRR используется год - рассмотрим целый год из данных - 2017
all = all.query ('year_approved == "2017.0"')

In [13]:
# Посчитаем частоту покупок клиентами (Frequency)
Frequency = all\
    .groupby ('customer_unique_id')\
    .agg ({'order_id': 'count'}).reset_index()
Frequency.head ()

,customer_unique_id,order_id
0,0000f46a3911fa3c0805444483337064,1
1,0000f6ccb0745a6a4b88665a16c9f078,1
2,0004aac84e0df4da2b147fca70cf8255,1
3,0005e1862207bf6ccc02e4228effd9a0,1
4,0006fdc98a402fceb4eb0ee528f6a8d4,1


In [14]:
# Рассчитаем сумму по покупкам для каждого пользователя за всё время (Monetary)
Monetary = all\
    .groupby(['customer_unique_id']).agg({'price': lambda x: x.sum()}).reset_index()
Monetary.head()

,customer_unique_id,price
0,0000f46a3911fa3c0805444483337064,69.00
1,0000f6ccb0745a6a4b88665a16c9f078,25.99
2,0004aac84e0df4da2b147fca70cf8255,180.00
3,0005e1862207bf6ccc02e4228effd9a0,135.00
4,0006fdc98a402fceb4eb0ee528f6a8d4,13.90


In [15]:
# Рассчитаем время от последней покупки пользователя до текущей даты (Recency)

In [15]:
# Добавим дату последней покупки + 1. Эта дата будет использоваться в качестве ориентира для расчета показателя недавности.
NOW = all ['order_approved_at'].max() + timedelta(days=1)
NOW

Timestamp('2018-01-01 23:32:40')

In [16]:
# Найдем месяц самого последнего заказа по клиентам
Recency = all\
    .groupby('customer_unique_id')\
    .agg ({'order_approved_at': 'max'})
Recency.head()

,order_approved_at
customer_unique_id,
0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03
0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:49:17
0004aac84e0df4da2b147fca70cf8255,2017-11-14 20:06:52
0005e1862207bf6ccc02e4228effd9a0,2017-03-04 23:43:26
0006fdc98a402fceb4eb0ee528f6a8d4,2017-07-18 09:30:22


In [17]:
# Добавим столбец с количеством дней между покупкой и настоящим моментом
Recency['DaysSinceOrder'] = Recency['order_approved_at'].apply(lambda x: (NOW - x).days)

In [18]:
Recency.head ()

,order_approved_at,DaysSinceOrder
customer_unique_id,,
0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,297
0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:49:17,81
0004aac84e0df4da2b147fca70cf8255,2017-11-14 20:06:52,48
0005e1862207bf6ccc02e4228effd9a0,2017-03-04 23:43:26,302
0006fdc98a402fceb4eb0ee528f6a8d4,2017-07-18 09:30:22,167


In [20]:
#объединим три получившиеся таблицы
FM = Frequency.merge(Monetary, how = 'outer', on ='customer_unique_id')
RFM = FM.merge(Recency, how = 'outer', on ='customer_unique_id')
RFM.rename(columns={'DaysSinceOrder': 'Недавность', 'order_id': 'Частота', 'price': 'Монетизация'}, inplace=True)
RFM.drop(columns=['order_approved_at'], inplace=True)

In [21]:
RFM.head ()

,customer_unique_id,Частота,Монетизация,Недавность
0,0000f46a3911fa3c0805444483337064,1,69.00,297
1,0000f6ccb0745a6a4b88665a16c9f078,1,25.99,81
2,0004aac84e0df4da2b147fca70cf8255,1,180.00,48
3,0005e1862207bf6ccc02e4228effd9a0,1,135.00,302
4,0006fdc98a402fceb4eb0ee528f6a8d4,1,13.90,167


In [23]:
RFM.groupby ('Частота').agg ({'customer_unique_id': 'count'})

,customer_unique_id
Частота,
1,37531
2,3990
3,754
4,306
5,100
6,75
7,12
8,10
9,3


При проверке мы видим, что кол-во клиентов, которые приобрели товар только 1 раз, превалирует в разы над остальными значениями.
Можем сделать вывод, что данный показатель не будет являться репрезентативным.
Но еще  проверим ниже на квантилях.

In [25]:
# Используем квантили для разбивки
quintiles = RFM[['Недавность', 'Частота', 'Монетизация']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'Недавность': {0.2: 38.0, 0.4: 92.0, 0.6: 156.0, 0.8: 232.0},
 'Частота': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'Монетизация': {0.2: 39.9, 0.4: 68.9, 0.6: 108.0, 0.8: 179.9}}

Видим, что у частоты квантили везде 1. Из-за явного неравномерного распределения все сильно смещается к числу покупок = 1.

Теперь запиишем функцию для присвоения рангов от 1 до 5. 
Чем меньше значение недавности, тем лучше. Чем больше частота и монетизация, тем лучше.

In [26]:
def r_score(x):
    if x <= quintiles['Недавность'][0.2]:
        return 5
    elif x <= quintiles['Недавность'][0.4]:
        return 4
    elif x <= quintiles['Недавность'][0.6]:
        return 3
    elif x <= quintiles['Недавность'][0.8]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quintiles[c][0.2]:
        return 1
    elif x <= quintiles[c][0.4]:
        return 2
    elif x <= quintiles[c][0.6]:
        return 3
    elif x <= quintiles[c][0.8]:
        return 4
    else:
        return 5    

In [27]:
# присвоим рейтинги
RFM['R'] = RFM['Недавность'].apply(lambda x: r_score(x))
RFM['F'] = RFM['Частота'].apply(lambda x: fm_score(x, 'Частота'))
RFM['M'] = RFM['Монетизация'].apply(lambda x: fm_score(x, 'Монетизация'))

In [28]:
# И создадим столбец RFM и RM
RFM['RFM Score'] = RFM['R'].map(str) + RFM['F'].map(str) + RFM['M'].map(str)
RFM

,customer_unique_id,Частота,Монетизация,Недавность,R,F,M,RFM Score
0,0000f46a3911fa3c0805444483337064,1,69.00,297,1,1,3,113
1,0000f6ccb0745a6a4b88665a16c9f078,1,25.99,81,4,1,1,411
2,0004aac84e0df4da2b147fca70cf8255,1,180.00,48,4,1,5,415
3,0005e1862207bf6ccc02e4228effd9a0,1,135.00,302,1,1,4,114
4,0006fdc98a402fceb4eb0ee528f6a8d4,1,13.90,167,2,1,1,211
...,...,...,...,...,...,...,...,...
42791,fffbf87b7a1a6fa8b03f081c5f51a201,1,149.00,5,5,1,4,514
42792,fffcf5a5ff07b0908bd4e2dbc735a684,2,1570.00,207,2,5,5,255
42793,fffea47cd6d3cc0a88bd621562a9d061,1,64.89,22,5,1,2,512
42794,ffff371b4d645b6ecea244b27531430a,1,89.90,328,1,1,3,113


Если посмотреть детально, то в частоте у нас есть значения 1 и 5. В сегмент 5 попадают все, кто сделал больше 1 покупки.

In [30]:
count = RFM. groupby('RFM Score').agg ({'customer_unique_id':'count'}).reset_index()
count.shape

(50, 2)

In [31]:
# Всего получилось 49 сегментов.

In [ ]:
{'Недавность': {0.2: 38.0, 0.4: 92.0, 0.6: 156.0, 0.8: 232.0},
 'Частота': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'Монетизация': {0.2: 39.9, 0.4: 68.9, 0.6: 108.0, 0.8: 179.9}}

Ответ:

Использовали квантили для разделения:
Недавность (R): 1: от 0 до 38 дней, 2: от 39 до 92 дней, 3: от 93 до 156 дней, 4: от 157 до 232 дней, 5: от 233 дней и выше
Частота (F): 1: заказчик сделал одну покупку, 2: -, 3: -, 4: -, 5: заказчик сделал больше одной покупки
Монетизация (M): 1: от 0 до 39.9, 2: от 39.9 (не вкл) до 68.9, 3: от 68.9 (не вкл) до 108.4: от 108.1 (не вкл) до 179.9: от 179.9 (не вкл) и выше}}
                        

Так как сегментов 49 и описывать все имеющиеся - долго и объемно, то приведу для примера 3 из них.

RFM-сегмент 413 (recency=4, frequency=1, monetary=3) имеет границы метрик recency от 15 до 232 дней,
frequency 1 заказ в неделю, monetary от 68.9 до 108.4 рублей 

RFM-сегмент 351 (recency=3, frequency=5, monetary=1) имеет границы метрик recency от 93 до 156 дней,
frequency больше 1 заказа в неделю, monetary от 0 до 39.9 рублей. 

RFM-сегмент 115 (recency=1, frequency=1, monetary=5) имеет границы метрик recency от 0 до 39 дней,
frequency 1 заказ в неделю, monetary от 179.9 и выше рублей. 